# Install Dependencies via

pip install -r requirements.txt

# Verify dash installation

In [1]:
import dash
print(dash.__version__)


2.17.1


# Load Product Invetory Data

In [2]:
import pandas as pd

# Relative path to the CSV file
productInventory = pd.read_csv('productInventory.csv')

# Display the first few rows of the DataFrame
print(productInventory.head())

   unq_id  loc_id  brand_id  prod_type_id  \
0  150001     452        11            19   
1  150002     452        16            23   
2  150003     452        16            25   
3  150004     452        16            29   
4  150005     452        16            34   

                                loc_nam loc_st  \
0  Village Green Society - Recreational     CO   
1  Village Green Society - Recreational     CO   
2  Village Green Society - Recreational     CO   
3  Village Green Society - Recreational     CO   
4  Village Green Society - Recreational     CO   

                           prod_name  prod_attribute prod_thc_lvl prod_qty  \
0             Cigarillos edible mint       Chocolate       1000mg       10   
1       East West concentrate budder      White Dawg          0.6      .5g   
2  East West concentrate cured resin    Thick Wicket         0.57       1g   
3         East West concentrate kief  Gorilla Snacks         0.57      .5g   
4   East West concentrate live jelly  

# Display Product Inventory Data info

In [3]:
# Display dataFrame info
print(productInventory.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   unq_id          150000 non-null  int64 
 1   loc_id          150000 non-null  int64 
 2   brand_id        150000 non-null  int64 
 3   prod_type_id    150000 non-null  int64 
 4   loc_nam         150000 non-null  object
 5   loc_st          150000 non-null  object
 6   prod_name       150000 non-null  object
 7   prod_attribute  150000 non-null  object
 8   prod_thc_lvl    150000 non-null  object
 9   prod_qty        150000 non-null  object
 10  prod_lst_price  150000 non-null  object
 11  loc_qty         150000 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 13.7+ MB
None


In [4]:
from dash import dcc, html, Dash, dash_table
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import pandas as pd

# Initialize the Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.COSMO])

# Assuming productInventory is your DataFrame
# Concatenate 'loc_id' and 'loc_nam' for the dropdown options
productInventory['loc_id_nam'] = productInventory['loc_id'].astype(str) + " - " + productInventory['loc_nam']

# Layout components
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Inventory Management Dashboard"), width=12),
    ], className="mb-4"),
    
    # Location and Threshold Input with Labels
    dbc.Row([
        dbc.Col([
            dbc.Label("Location"),
            dcc.Dropdown(
                id='location-dropdown',
                options=[{'label': loc, 'value': loc} for loc in productInventory['loc_id_nam'].unique()],
                placeholder="Select a location",
                value=productInventory['loc_id_nam'].unique()[0],  # Default to first location
                className="text-info"  # Ensures text is dark mode friendly
            ),
        ], width=6, className="mb-3"),
        dbc.Col([
            dbc.Label("Quantity Threshold"),
            dcc.Input(
                id='threshold-input',
                type='number',
                placeholder='Set inventory threshold',
                value=10,  # Default threshold value
                className="mt-2",
            ),
        ], width=6, className="mb-3"),
    ], className="mb-4"),
    
    # Location Inventory Overview
    dbc.Row([
        dbc.Col(dash_table.DataTable(id='location-inventory-table'), width=12),
    ], className="mb-4"),
])

# Function to update Location Inventory Overview based on selected location and threshold
@app.callback(
    Output('location-inventory-table', 'data'),
    Output('location-inventory-table', 'columns'),
    [Input('location-dropdown', 'value'),
     Input('threshold-input', 'value')]
)
def update_location_inventory_table(selected_location, threshold):
    # Extract loc_id and loc_nam from the selected location value
    loc_id = selected_location.split(" - ")[0]
    
    # Filter the data based on the selected location and threshold
    filtered_data = productInventory[(productInventory['loc_id'].astype(str) == loc_id) & 
                                     (productInventory['loc_qty'] < threshold)]
    
    # Select only the columns of interest
    filtered_data = filtered_data[['loc_st', 'prod_name', 'prod_attribute', 'prod_thc_lvl', 
                                   'prod_qty', 'prod_lst_price', 'loc_qty']]
    
    # Prepare the data for the DataTable
    data = filtered_data.to_dict('records')
    columns = [{"name": i, "id": i} for i in filtered_data.columns]

    return data, columns

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



